In [1]:


import pandas as pd

import plotly.io as pio

pio.get_chrome()

2025-10-16 23:40:49.561 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


PosixPath('/home/jaden/Sources/followchon_back/.venv/lib/python3.12/site-packages/choreographer/cli/browser_exe/chrome-linux64/chrome')

In [6]:
df_cities = pd.read_csv(
    '../data/formated/cities-france.csv',
    dtype={'code_insee': str, 'dep_code': str, 'nom_standard': str, 'latitude_mairie': float, 'longitude_mairie': float},
).rename(columns={'code_insee': 'index', 'nom_standard': 'name', 'latitude_mairie': 'lat', 'longitude_mairie': 'lng'}) \
    .set_index('index') \
    .loc[:, ['lat', 'lng']]

/home/jaden/Sources/followchon_back/.venv/lib/python3.12/site-packages/sklearn/manifold/_mds.py:677: FutureWarning:

The default value of `n_init` will change from 4 to 1 in 1.9.



Step 1 with 70 points
- with 23 clusters with 9 max size
- with 24 clusters with 6 max size
- with 25 clusters with 7 max size
- with 26 clusters with 6 max size
- with 27 clusters with 7 max size
- with 28 clusters with 6 max size
- with 29 clusters with 6 max size
- with 30 clusters with 5 max size
- with 31 clusters with 6 max size
- with 32 clusters with 6 max size
- with 33 clusters with 6 max size
- with 34 clusters with 5 max size
- with 35 clusters with 7 max size
- with 36 clusters with 6 max size
- with 37 clusters with 5 max size
- with 38 clusters with 6 max size
- with 39 clusters with 6 max size
- with 40 clusters with 5 max size
- with 41 clusters with 4 max size
- with 42 clusters with 4 max size
- with 43 clusters with 6 max size
- with 44 clusters with 5 max size
- with 45 clusters with 4 max size
- with 46 clusters with 4 max size
- with 47 clusters with 4 max size
- with 48 clusters with 5 max size
- with 49 clusters with 4 max size
- with 50 clusters with 4 max siz

/home/jaden/Sources/followchon_back/.venv/lib/python3.12/site-packages/sklearn/manifold/_mds.py:677: FutureWarning:

The default value of `n_init` will change from 4 to 1 in 1.9.



Step 2 with 53 points
- with 17 clusters with 6 max size
- with 18 clusters with 7 max size
- with 19 clusters with 6 max size
- with 20 clusters with 5 max size
- with 21 clusters with 4 max size
- with 22 clusters with 5 max size
- with 23 clusters with 4 max size
- with 24 clusters with 4 max size
- with 25 clusters with 4 max size
- with 26 clusters with 4 max size
- with 27 clusters with 4 max size
- with 28 clusters with 5 max size
- with 29 clusters with 5 max size
- with 30 clusters with 4 max size
- with 31 clusters with 4 max size
- with 32 clusters with 4 max size
- with 33 clusters with 4 max size
- with 34 clusters with 4 max size
- with 35 clusters with 3 max size


/home/jaden/Sources/followchon_back/.venv/lib/python3.12/site-packages/sklearn/manifold/_mds.py:677: FutureWarning:

The default value of `n_init` will change from 4 to 1 in 1.9.



...

Step 4 with 18 points
- with 6 clusters with 5 max size
- with 7 clusters with 3 max size


/home/jaden/Sources/followchon_back/.venv/lib/python3.12/site-packages/sklearn/manifold/_mds.py:677: FutureWarning:

The default value of `n_init` will change from 4 to 1 in 1.9.



Step 5 with 7 points
- with 3 clusters with 3 max size


/home/jaden/Sources/followchon_back/.venv/lib/python3.12/site-packages/sklearn/manifold/_mds.py:677: FutureWarning:

The default value of `n_init` will change from 4 to 1 in 1.9.



Step 6 with 3 points
- with 1 clusters with 3 max size


In [8]:
steps_list = range(2, steps_count + 1)[::-1]
print("steps :", list(steps_list))
df_tour = pd.DataFrame()

n = steps_list[0]
df_centroids = pd.read_csv('data/step_' + str(n) + '_points.csv', dtype={'index': str, 'cluster': str})

for n in steps_list:
    df_cluster = None

    df_centroids_distances = pd.read_csv('data/step_' + str(n) + '_distances.csv', dtype={'index': str})
    df_centroids_distances = df_centroids_distances.set_index('index')

    print(f'Begin layer {n - 1} with {df_centroids.shape[0]} points')

    df_points = pd.read_csv('data/step_' + str(n - 1) + '_points.csv', dtype={'index': str, 'cluster': str})

    df_points_distances = pd.read_csv('data/step_' + str(n - 1) + '_distances.csv', dtype={'index': str})
    df_points_distances = df_points_distances.set_index('index')

    if df_centroids.shape[0] < calcul_max_clusters:
        df_cluster = best_path(df_centroids, df_points, df_points_distances)

    if df_cluster is None:
        df_cluster = simple_path(df_centroids, df_points, df_points_distances)

    draw_map(df_cluster, True).save(f'export/step_{n - 1}_map.html')
    df_centroids = df_cluster[['index', 'lat', 'lng', 'x', 'y', 'cluster']]

    print(f'End layer {n - 1} with {df_centroids.shape[0]} points')

steps : [6, 5, 4, 3, 2]
Begin layer 5 with 3 points
- [Step 1] with 3 clusters
- [Step 2] with 24 combinaisons
- [Step 3]
End layer 5 with 7 points
Begin layer 4 with 7 points
- [Step 1] with 7 clusters
- [Step 2] with 15552 combinaisons
- [Step 3]
End layer 4 with 18 points
Begin layer 3 with 18 points
- [Step 1] with 18 clusters
- [Step 2] with 1492992 combinaisons
- [Step 3] with 35 points and distance min 5464.0
End layer 3 with 35 points
Begin layer 2 with 35 points
- [Step 1] with 35 clusters
- [Step 2] with 1990656 combinaisons
- [Step 3] with 53 points and distance min 5564.0
End layer 2 with 53 points
Begin layer 1 with 53 points
- [Step 3] with 70 points and distance min 5757.0
End layer 1 with 70 points
